In [3]:
from tensorflow.keras import datasets
import matplotlib.pyplot as plt
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.metrics import classification_report

### RNNs

* Redes recorrentes são utilizadas para lidar com dados sequenciais, como por exemplo:
  * Previsão de séries temporais;
  * Reconhecimento de fala;
  * Tradução de idiomas;
  * Reconhecimento de ações em vídeo (como "rapaz correndo", "moça tocando violão");
  * Geração de música;
  * Problemas de genética, que envolvem sequenciamentos de DNAs
  
* Para lidar com esse tipo de dados, RNNs tradicionais utilizam uma estrutuar de neurônios de forma semelhante a uma MLP, adicionando uma estrutura de retroalimentação (como um loop) para aprender as informações de cada dado incrementalmente;

 ![Title](imgs/rnns.png) 
 
* Entretanto, essa estrutura não é muito eficiente para longas sequências devido a problemas do gradiente (vanish gradient)

![Title](imgs/gradient_rnns.gif) 
 
* Redes LSTM, por outro lado, são redes recorrentes que filtram a entrada por meio de uma estrutura composta por gates e célula, de forma a reduzir os efeitos do gradiente para sequências maiores 
  *  Célula: conecta a entrada da célula a resposta de cada gate
  * Gates: funções de ativação que vão analisar a entrada e filtrar o conhecimento que deverá ser esquecido temporariamente para compreender o dado atual (forget gate), conhecimento que deverá ser adicionado para compreender o dado atual (update gate) e output gate é a parte da rede responsável pela tarefa propriamente dita para gerar a saída do modelo
 

### Leitura da base de dados

In [4]:
max_features = 20000 ### tamanho do vocabulário
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=max_features) 
### leitura da base de dados de treino e teste

17464789/17464789 [==============================] - 8s 0us/step


In [7]:
x_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 82

### Visualizando as sentenças 

In [8]:
word_index = datasets.imdb.get_word_index() ### dicionário com cada palavra do vocabulario
inverted_word_index = dict((i, word) for (word, i) in word_index.items()) ### pega chave e valor do dicionário para decodificar a sentença
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0]) ## desfaz a tokenização

In [9]:
decoded_sequence

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit the

In [10]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

### Pré-processamento

Uma rede neural, seja MLP, CNN ou RNNs, trabalham com entradas com duas características
* Tamanho fixo
* Entrada numérica

Sendo assim, como pre-processamento inicial, deve-se verificar que essas duas características estão atendidas antes de implementar uma RNN

In [11]:
# Quantidade de palavras
len(x_train[2])

141

In [16]:
maxlen = 200 ### tamanho maximo de cada amostra

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen) ### aplica o padding
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
### sentenças menores que 200 vão ser preenchidas com zero, sentenças maiores que 200 caracteres terão seus caracteres eliminados

In [17]:
x_treino, x_val, y_treino, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 13)

In [18]:
print(x_treino.shape)
print(x_val.shape)
print(x_teste.shape)

(22500, 200)
(2500, 200)
(25000, 200)


### Arquitetura de rede

In [20]:
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [21]:
x_treino.max() ### tamanho máximo de palavras no vocabulário

19999

In [22]:
x_treino

array([[   0,    0,    0, ...,   10,  693,  158],
       [  24,   17,   78, ...,  251,  342,  158],
       [   0,    0,    0, ...,  106,    5,  358],
       ...,
       [3137,    7,    6, ...,  736, 2929, 1359],
       [  18,    4, 1047, ..., 8533,   23, 1092],
       [   0,    0,    0, ..., 1681,  180,  133]])

In [23]:
rnn = Sequential() ### define um modelo como sequencial (cada camada terá como entrada o resultado da camada anterior)

rnn.add(Input(shape=(maxlen,))) ### define a entrada do modelo. A entrada é colocada de forma que cada amostra tenha o tamanho máximo definido anteriormente (200 caracteres)
rnn.add(Embedding(max_features, 128)) ### embedding é formada passando o tamanho máximo de vocabulario (20000) e o tamanho do vetor de saida (128). Com isso cada palavra do vocabulario terá um vetor de 128 posições indicando a relação dela com as palavras do vocabulário

rnn.add((LSTM(64))) ### cria uma lstm com 64 células
rnn.add(Dense(64, activation="relu")) ### cria uma camada completamente conectada de 64 neuronios e função de ativação relu
rnn.add(Dense(1, activation="sigmoid")) ### cria uma camada de saida para aclassificar o modelo
rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"]) ### define a otimização da rede neural
hist = rnn.fit(x_treino, y_treino, epochs=2, batch_size=32, validation_data=(x_val, y_val)) ### treina o modelo

Epoch 1/2
704/704 [==============================] - 85s 114ms/step - loss: 0.4128 - accuracy: 0.8081 - val_loss: 0.2894 - val_accuracy: 0.8840
Epoch 2/2
704/704 [==============================] - 91s 129ms/step - loss: 0.2041 - accuracy: 0.9229 - val_loss: 0.3106 - val_accuracy: 0.8764


## Podemos utilizar mais de uma LSTM?

Sim! Quanto mais complexo o problema, mais LSTMs precisamos!
Sendo assim, é comum criarmos diferentes LSTMs, uma recebendo o resultado do processamento da outra.
Para isso, basta colocarmos um rnn.add para o Keras e adicionar mais uma LSTM. Além disso, deverá setar o parâmetro return_sequences como True.

#### O que é o return sequences?

* Uma LSTM pode retornar tanto o processamento final dela (a resposta), como o aprendizado que teve
* Para classificar, só é necessário a resposta da LSTM
* Para utilizar duas LSTM em sequência, precisamos passar para a próxima LSTM os dois retornos, isso é feito utilizando o return_sequences

#### LSTMs bidirecionais

* Também podemos definir a LSTM de forma bidirecional
* Uma LSTM bidirecional possui dois sentidos
* Aprende a palavra atual baseado no contexto passado
* Aprende a palavra atual baseado no contexto do futuro 
* Útil em casos que a resposta está apenas mais a frente da frase. Por exemplo, quando sujeito e predicado estão invertidos 

In [22]:
rnn = Sequential()

rnn.add(Input(shape=(maxlen,)))
rnn.add(Embedding(max_features, 128))

rnn.add(Bidirectional(LSTM(64, return_sequences=True)))
rnn.add((LSTM(64)))
rnn.add(Dense(64, activation="relu"))
rnn.add(Dense(1, activation="sigmoid"))
rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
hist = rnn.fit(x_treino, y_treino, epochs=2, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/2
704/704 [==============================] - 229s 321ms/step - loss: 0.3923 - accuracy: 0.8207 - val_loss: 0.2838 - val_accuracy: 0.8868
Epoch 2/2
704/704 [==============================] - 240s 341ms/step - loss: 0.2060 - accuracy: 0.9223 - val_loss: 0.3648 - val_accuracy: 0.8648


In [28]:
scores = rnn.evaluate(x_teste, y_teste, verbose=1) ### faz uma avaliação geral do modelo indicando a loss e a acc
scores

782/782 [==============================] - 48s 61ms/step - loss: 0.3863 - accuracy: 0.8516


[0.3862696886062622, 0.8515599966049194]

In [29]:
y_pred=rnn.predict(x_teste) #### retorna todas as predições

In [41]:
y_pred ### saída não binária, necessário converter para 0 e 1 igual o y_test

array([[0.2644105 ],
       [0.99584997],
       [0.96135825],
       ...,
       [0.1864289 ],
       [0.12678808],
       [0.9022596 ]], dtype=float32)

In [31]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [40]:
print(classification_report(y_test, y_pred.round())) ### desempenho do modelo com sklearn

              precision    recall  f1-score   support

           0       0.91      0.78      0.84     12500
           1       0.81      0.92      0.86     12500

    accuracy                           0.85     25000
   macro avg       0.86      0.85      0.85     25000
weighted avg       0.86      0.85      0.85     25000



### Exemplo IDMB em raw - LSTM

In [50]:
import pandas as pd
df = pd.read_csv('../datasets/sentiment labelled sentences/imdb_labelled.txt', error_bad_lines=False, delimiter='\t', header=None)

/var/folders/t4/7csz5lt17fs3dd7hl2vvgns00000gn/T/ipykernel_9654/3060947599.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('../datasets/sentiment labelled sentences/imdb_labelled.txt', error_bad_lines=False, delimiter='\t', header=None)


In [51]:
df

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0


In [59]:
x = df[0]
y = df[1]
sentences_train, sentences_val, y_train, y_val = train_test_split(x, y, test_size = 0.1, random_state = 13)

In [60]:
tokenizer = Tokenizer(num_words=20000) ### cria o tokenizador indicando que o vocabulário será composto por 20000 palavras
tokenizer.fit_on_texts(sentences_train) ### passa as palavras para frequencia

X_train = tokenizer.texts_to_sequences(sentences_train) ### fita no treino
X_val = tokenizer.texts_to_sequences(sentences_val) ### fita no teste

vocab_size = len(tokenizer.word_index) + 1 

print(sentences_train[2])
print(X_train[2])

Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  
[29, 28, 170, 442, 1055, 1056, 2, 635, 1057, 50, 5, 32, 636, 27, 1, 39]


In [61]:
for word in ['the', 'all']:
    print('{}: {}'.format(word, tokenizer.word_index[word]))

the: 1
all: 27


In [62]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen) ### faz o padding
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)

In [63]:
max_features = 20000
rnn = Sequential()

rnn.add(Input(shape=(maxlen,)))
rnn.add(Embedding(max_features, 128))

rnn.add(Bidirectional(LSTM(64, return_sequences=True)))
rnn.add(Bidirectional(LSTM(64)))
rnn.add(Dense(64, activation="relu"))
rnn.add(Dense(1, activation="sigmoid"))
rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
hist = rnn.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/2
22/22 [==============================] - 11s 333ms/step - loss: 0.0000e+00 - accuracy: 0.4889 - val_loss: 0.0000e+00 - val_accuracy: 0.4800
Epoch 2/2
22/22 [==============================] - 7s 331ms/step - loss: 0.0000e+00 - accuracy: 0.4844 - val_loss: 0.0000e+00 - val_accuracy: 0.4800
